This notebook converts tabular data to graph data for pytorch geometric to ingest.

The final graph is a static homogeneous directed graph with temporal signals.


In [28]:
import math
import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch_geometric.nn import GATv2Conv
from torch_geometric.data import Data
from torch.nn import BatchNorm1d

import pandas as pd
import numpy as np

from tqdm import tqdm

In [29]:
flow_df = pd.read_csv("data/flow/_combined.csv")
dap_df = pd.read_csv("data/dap/_combined.csv")
flow_df.set_index("datetime", inplace=True)
dap_df.set_index("datetime", inplace=True)

datetime_intersect = flow_df.index.intersection(dap_df.index)
flow_df = flow_df.loc[datetime_intersect]
dap_df = dap_df.loc[datetime_intersect]

# Remove columns that contains "UK" or "IE"
flow_df = flow_df.loc[
    :, ~flow_df.columns.str.contains("UK") & ~flow_df.columns.str.contains("IE")
]
dap_df = dap_df.loc[
    :, ~dap_df.columns.str.contains("UK") & ~dap_df.columns.str.contains("IE")
]

# print(flow_df.isnull().sum())
# print(dap_df.isnull().sum())
assert not flow_df.isnull().values.any()
assert not dap_df.isnull().values.any()
dap_df = dap_df.reindex(sorted(dap_df.columns), axis=1)

In [30]:
# Normalize
# is this good enough?
flow_df = (flow_df - flow_df.mean()) / flow_df.std()
dap_df = (dap_df - dap_df.mean()) / dap_df.std()

# print(flow_df.head())
# print(dap_df.head())

In [31]:
# Static edges of shape (2, n_edges)
interconnectors = flow_df.columns[1:]
exporters = []
importers = []
for ic in interconnectors:
    exporter, importer = ic.split("->")
    exporters.append(exporter)
    importers.append(importer)

exporters = np.array(exporters)
importers = np.array(importers)

edges = np.vstack([exporters, importers])
print(edges.shape)
print(edges[:, :10])
n_edges = edges.shape[1]

(2, 45)
[['BE' 'BE' 'BE' 'DE' 'DE' 'DE' 'DE' 'DE' 'DE' 'DE']
 ['FR' 'LU' 'NL' 'BE' 'DK' 'AT' 'CH' 'CZ' 'FR' 'LU']]


In [32]:
# Map edge names to indices
edge_names = np.unique(edges)
edge_map = {edge: i for i, edge in enumerate(edge_names)}
edge_indices = np.array([edge_map[edge] for edge in edges.flatten()]).reshape(
    edges.shape
)
# Repeat edge indices for each datetime
edge_indices = np.repeat(
    edge_indices[np.newaxis, :, :],
    len(datetime_intersect),
    axis=0,
)
print(edge_indices.shape)

(5136, 2, 45)


In [33]:
# Edge labels (flow) of shape (n_datetimes, n_edges, 1)
edge_labels = np.array(flow_df[interconnectors])
edge_labels = np.reshape(edge_labels, (edge_labels.shape[0], edge_labels.shape[1], 1))
print(edge_labels.shape)

(5136, 45, 1)


In [34]:
# Edge attributes (capacity, etc.) of shape (n_datetimes, n_edges, n_attributes)
# copy the edge labels to the edge attributes
edge_attributes = np.copy(edge_labels)
# hard code the edge attributes to be ones for now
edge_attributes = np.ones(edge_labels.shape)
print(edge_attributes.shape)
# print(edge_attributes[:, :, 0])

(5136, 45, 1)


In [35]:
# Node features (dap)
node_features = np.array(dap_df)
node_features = np.array(node_features)
node_features = np.reshape(

    node_features, (node_features.shape[0], node_features.shape[1], 1)
)

print(node_features.shape)

n_nodes = node_features.shape[1]

(5136, 15, 1)


In [36]:
assert (
    len(datetime_intersect)
    == edge_indices.shape[0]
    == edge_attributes.shape[0]
    == edge_labels.shape[0]
    == node_features.shape[0]
)
# Print a snapshot of the shape of the graph data
i = 256
print("Edge indices:", edge_indices[i].shape)
print("Edge attributes:", edge_attributes[i].shape)
print("Edge labels:", edge_labels[i].shape)
print("Node features:", node_features[i].shape)

Edge indices: (2, 45)
Edge attributes: (45, 1)
Edge labels: (45, 1)
Node features: (15, 1)


In [37]:
n_snapshots = len(datetime_intersect)
snapshots = []
for i in range(n_snapshots):
    data = Data(
        x=torch.tensor(node_features[i], dtype=torch.float),
        edge_index=torch.tensor(edge_indices[i], dtype=torch.long),
        edge_attr=torch.tensor(edge_attributes[i], dtype=torch.float),
        y=torch.tensor(edge_labels[i], dtype=torch.float),
    )
    snapshots.append(data)
print(snapshots[0])

Data(x=[15, 1], edge_index=[2, 45], edge_attr=[45, 1], y=[45, 1])


In [38]:
# https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.nn.conv.GATv2Conv.html
class GNNEncoder(nn.Module):
    def __init__(
        self, hidden_channels, num_heads_GAT, dropout_p_GAT, edge_dim_GAT, momentum_GAT
    ):
        super().__init__()
        self.gat = GATv2Conv(
            (-1, -1),
            hidden_channels,
            add_self_loops=False,
            heads=num_heads_GAT,
            edge_dim=edge_dim_GAT,
        )
        self.norm = BatchNorm1d(
            hidden_channels,
            momentum=momentum_GAT,
            affine=False,
            track_running_stats=False,
        )
        self.dropout = nn.Dropout(dropout_p_GAT)

    def forward(self, x, edge_indices, edge_attrs):
        x = self.dropout(x)
        x = self.norm(x)
        nodes_embedds = self.gat(x, edge_indices, edge_attrs)
        nodes_embedds = F.leaky_relu(nodes_embedds, negative_slope=0.1)
        return nodes_embedds

In [39]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model)
        )
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer("pe", pe)

    def forward(self, x: Tensor) -> Tensor:
        x = x + self.pe[: x.size(0)]
        return self.dropout(x)


class Transformer(nn.Module):
    """
    Transformer-based module for creating temporal node embeddings.

    Args:
        dim_model (int): The dimension of the model's hidden states.
        num_heads_TR (int): The number of attention heads.
        num_encoder_layers_TR (int): The number of encoder layers.
        num_decoder_layers_TR (int): The number of decoder layers.
        dropout_p_TR (float): Dropout probability.
    """

    def __init__(
        self,
        dim_model,
        num_heads_TR,
        num_encoder_layers_TR,
        num_decoder_layers_TR,
        dropout_p_TR,
    ):
        super().__init__()
        self.pos_encoder = PositionalEncoding(dim_model)
        self.transformer = nn.Transformer(
            d_model=dim_model,
            nhead=num_heads_TR,
            num_decoder_layers=num_encoder_layers_TR,
            num_encoder_layers=num_decoder_layers_TR,
            dropout=dropout_p_TR,
        )

    def forward(self, src, trg):
        src = self.pos_encoder(src)
        trg = self.pos_encoder(trg)
        temporal_node_embeddings = self.transformer(src, trg)
        return temporal_node_embeddings

In [40]:
class EdgeDecoder(nn.Module):
    def __init__(self, hidden_channels, num_heads_GAT, num_edges, num_nodes):
        super().__init__()
        self.lin1 = nn.Linear(
            num_nodes * hidden_channels * num_heads_GAT, hidden_channels
        )
        self.lin2 = nn.Linear(hidden_channels, num_edges)

    def forward(self, x):
        # Flatten the tensor
        x = torch.flatten(x)
        x = self.lin1(x)
        x = F.leaky_relu(x, negative_slope=0.1)
        x = self.lin2(x)
        return x.view(-1)

In [41]:
class Model(nn.Module):

    def __init__(
        self,
        hidden_channels,
        num_heads_GAT,
        dropout_p_GAT,
        edge_dim_GAT,
        momentum_GAT,
        dim_model,
        num_heads_TR,
        num_encoder_layers_TR,
        num_decoder_layers_TR,
        dropout_p_TR,
        n_edges,
        n_nodes,
    ):
        super().__init__()
        self.encoder = GNNEncoder(
            hidden_channels, num_heads_GAT, dropout_p_GAT, edge_dim_GAT, momentum_GAT
        )
        self.transformer = Transformer(
            dim_model,
            num_heads_TR,
            num_encoder_layers_TR,
            num_decoder_layers_TR,
            dropout_p_TR,
        )
        self.decoder = EdgeDecoder(hidden_channels, num_heads_GAT, n_edges, n_nodes)

    def forward(self, x, edge_indices, edge_attrs):
        src_embedds = []
        for i in range(x.shape[0]):
            src_embedds.append(self.encoder(x[i], edge_indices[i], edge_attrs[i]))
        src_embedds = torch.stack(src_embedds)
        trg_embedds = src_embedds[-1].unsqueeze(0)
        temporal_node_embedds = self.transformer(src_embedds, trg_embedds)
        temporal_node_embedds = temporal_node_embedds.squeeze(0)
        edge_predictions = self.decoder(temporal_node_embedds)
        return edge_predictions

In [42]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Model(
    hidden_channels=8,
    num_heads_GAT=2,
    dropout_p_GAT=0.1,
    edge_dim_GAT=1,  # edge attributes
    momentum_GAT=0.1,
    dim_model=hidden_channels * num_heads_GAT,
    num_heads_TR=2,
    num_encoder_layers_TR=2,
    num_decoder_layers_TR=2,
    dropout_p_TR=0.1,
    n_edges=n_edges,
    n_nodes=n_nodes,
)
model = model.to(device)

c:\Users\lemuelkl\anaconda3\envs\euics\Lib\site-packages\torch\nn\modules\transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [43]:
n_epochs = 10
for epoch in range(n_epochs):
    window = 24 * 7
    for m in tqdm(range(window, len(snapshots))):
        history = snapshots[m - window : m]
        y = snapshots[m].y.view(-1)

        optimizer = torch.optim.Adam(model.parameters(), lr=0.00005)
        optimizer.zero_grad()

        x = [data.x for data in history]
        edge_indices = [data.edge_index for data in history]
        edge_attrs = [data.edge_attr for data in history]

        x = torch.stack(x)
        edge_indices = torch.stack(edge_indices)
        edge_attrs = torch.stack(edge_attrs)
        y = y.to(device)

        x = x.to(device)
        edge_indices = edge_indices.to(device)
        edge_attrs = edge_attrs.to(device)

        edge_predictions = model(x, edge_indices, edge_attrs)

        loss = F.mse_loss(edge_predictions, y)
        loss.backward()
        optimizer.step()

        if (m - window) % 100 == 0:
            print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

  0%|          | 2/4968 [00:00<18:15,  4.53it/s]

Epoch 1, Loss: 1.0799682140350342


  1%|          | 25/4968 [00:05<16:35,  4.96it/s]